<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Pandas for Exploratory Data Analysis: Joining Data

_Authors: Kevin Markham (DC)_

In [1]:
import pandas as pd

In [2]:
DATA_DIR = '../assets/data'

## Concatenating DataFrames

Concatenate sticks dataframes together, either on top of each other or next to each other.

In [3]:
# Define a DataFrame
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df1.head()

,letter,number
0,a,1
1,b,2


In [4]:
# Define a second DataFrame
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])
df2.head()

,letter,number
0,c,3
1,d,4


In [5]:
# Stick the dataframes on top of each other
# /scrub/
pd.concat([df1, df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [6]:
# Stick the dataframes next to each other.
# /scrub/
pd.concat([df1, df2], axis='columns')

,letter,number,letter,number
0,a,1,c,3
1,b,2,d,4


## Joining (Merging) DataFrames

### Example

"Tidy data" is data that is organized as follows:

- One variable per column
- One observation per row
- One type of observational unit per table

In [7]:
# Load movie titles
movie_cols = ['movie_id', 'title']
movies_path = f'{DATA_DIR}/movies.tbl'
movies = pd.read_csv(movies_path, sep='|', header=None, names=movie_cols, usecols=[0, 1], encoding='latin-1')
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
# Load movie ratings
rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_path = f'{DATA_DIR}/movie_ratings.tsv'
ratings = pd.read_csv(ratings_path, sep='\t', header=None, names=rating_cols)
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Now if we want to know the title associated with a particular rating, we match up the rows of the two tables by `movie_id`.

SQL calls this "matching up" operation a **join**. Pandas has a method called "join," but it also has a more flexible method called **merge**.

In [9]:
# merge 'movies' and 'ratings' (inner join on 'movie_id')
# /scrub/
movie_ratings = movies.merge(ratings)
movie_ratings.head()

,movie_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324


In [10]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

(1682, 2)
(100000, 4)
(100000, 5)


Datasets are often stored in tidy format because that format is space-efficient, flexible, and conceptually clean. However, it may require you to do multiple joins to get all of the information you want for your model in one place.

### Selecting Columns to Merge On

In [11]:
# Create sample DataFrame
df1 = pd.DataFrame([['a', 1], ['b', 2], ['c', 3], ['d', 4]], columns=['letter', 'number'])
df1

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


In [12]:
# Create sample DataFrame
df2 = pd.DataFrame([['e', 2], ['f', 3]], columns=['letter', 'number'])
df2

,letter,number
0,e,2
1,f,3


In [13]:
# Create sample DataFrame
df3 = pd.DataFrame([['f', 3], ['g', 4]], columns=['character', 'number'])
df3

,character,number
0,f,3
1,g,4


I recommend opening a second view of this notebook so that you can keep `df1`, `df2`, and `df3` in view.

By default, `merge` matches up rows that have the same values for columns that are common between the two tables.

In [14]:
# Join `df2` to `df1` on all columns
# /scrub/
df1.merge(df2)

,letter,number


In this case, `df1` and `df2` both have "letter" and "number" columns, but none of their rows have the same values for both of those columns.

You can override this default behavior of `merge` by telling it exactly which columns to use.

In [15]:
# Join `df2` to `df1` on the "number" column
# /scrub/
df1.merge(df2, on=['number'])

,letter_x,number,letter_y
0,b,2,e
1,c,3,f


Notice that `merge` adds suffixes (here "\_x" and "\_y") to the names of the columns from `df1` and `df2`, respectively, that are common between `df1` and `df2` but were not used in the join.

You can also tell it to match on columns with different names.

In [16]:
# Join `df3` to `df2` matching the "character" coulumn
# of `df3 to the "letter" column of `df2`.
# /scrub/
df2.merge(df3, left_on='letter', right_on='character')

,letter,number_x,character,number_y
0,f,3,f,3


You can also tell it to merge on the indices.

In [17]:
# Join `df2` to `df1` on their indices
# /scrub/
df1.merge(df2, left_index=True, right_index=True)

,letter_x,number_x,letter_y,number_y
0,a,1,e,2
1,b,2,f,3


### Join Types

You might have noticed that when we used `merge` above, we dropped rows from the original DataFrames that weren't matched. That kind of join is called an "inner join." You can also do an "other join", where rows that don't have a match are returned with missing values for the fields that would have come from the other table if there had been a match.

In [30]:
# Do an outer join of `df3` to `df2`
# /scrub/
df2.merge(df3, on=['number'], how='outer')

,letter,number,character
0,e,2,NaN
1,f,3,f
2,NaN,4,g


You can also do a "left merge" that returns all rows from the "left" DataFrame only regardless of whether they have a match or a "right merge" that does likewise for the "right" DataFrame.

In [19]:
# Join `df3` to `df2` on "number", keeping all rows from `df2`
# even if they don't have a match.
# /scrub/
df2.merge(df3, on=['number'], how='left')

,letter,number,character
0,e,2,NaN
1,f,3,f


In [20]:
# Join `df3` to `df2` on "number", keeping all rows from `df3`
# even if they don't have a match.
# /scrub/
df2.merge(df3, on=['number'], how='right')

,letter,number,character
0,f,3,f
1,NaN,4,g


![](../assets/images/join_types.png)

In [21]:
# Create some more realistic data, on stock prices
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})

In [22]:
# Join the `openprice` and `wkhigh` dataframes together.
# /scrub/
openprice.merge(wkhigh, how='left', left_on='Symbol', right_on='Symbol')

,Symbol,OpenPrice,52wkHigh
0,AAPL,217.51,233.47
1,DHR,96.54,110.11
2,DAL,51.45,60.79
3,AMZN,1703.34,2050.49


**Exercise (6 mins., in groups)**

- Why do you think the values in the "Symbol" column in the merge result are in the order that they are in?

/scrub/

They take their ordering from the "left" DataFrame.

- **BONUS:** Find a way to test your answer to the previous question.

In [23]:
# /scrub/
wkhigh.merge(openprice, how='left', left_on='Symbol', right_on='Symbol')

,Symbol,52wkHigh,OpenPrice
0,DAL,60.79,51.45
1,AMZN,2050.49,1703.34
2,AAPL,233.47,217.51
3,DHR,110.11,96.54


- Join the `openprice` and `stockname` dataframes and inspect the result.

In [24]:
# /scrub/
openprice.merge(stockname, how='left', left_on='Symbol', right_on='Symbol')

,Symbol,OpenPrice,Name
0,AAPL,217.51,Apple
1,DHR,96.54,Danaher
2,DAL,51.45,Delta Airlines
3,AMZN,1703.34,Amazon


- Join all three dataframes together and inspect the result.

In [25]:
# /scrub/
(openprice.merge(stockname, how='left', left_on='Symbol', right_on='Symbol')
 .merge(wkhigh, how='left', left_on='Symbol', right_on='Symbol')
)

,Symbol,OpenPrice,Name,52wkHigh
0,AAPL,217.51,Apple,233.47
1,DHR,96.54,Danaher,110.11
2,DAL,51.45,Delta Airlines,60.79
3,AMZN,1703.34,Amazon,2050.49


$\blacksquare$

## Sales Data Example

Here are the data dictionaries we'll be using for the following exercise:

<ul>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html">Production.Product</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubcategory.html">Production.ProductSubcategory</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html">Sales.SalesOrderHeader</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html">Sales.SalesOrderDetail</a></li>
</ul>

In [26]:
# Load the data
product = pd.read_csv(f'{DATA_DIR}/Production.Product.csv', sep='\t')
product_subcategory = pd.read_csv(f'{DATA_DIR}/Production.ProductSubcategory.csv', sep='\t')
sales_header = pd.read_csv(f'{DATA_DIR}/Sales.SalesOrderHeader.csv', sep='\t', nrows=1000)
sales_detail = pd.read_csv(f'{DATA_DIR}/Sales.SalesOrderDetail.csv', sep='\t', nrows=1000)

The [Production.ProductSubcategory data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubcategory.html) reports a "primary key" `ProductSubcategoryID`. The fact that this field is designated the "primary key" means that its values are unique within the table and it is considered the primary identifier for items in that table. In traditional relational databases, every table has exactly one primary key.

The [Production.Product data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html) characerizes `ProductSubcategoryID` as a "foreign key." The fact that this field is designated a "foreign key" means that it corresponds to another table's primary key -- in this case, the primary key of `Production.ProductSubcategory`. This field is in the `Production.Product` table exactly so that we can join it to `Production.ProductSubcategory`. There are no restrictions on the number of foreign keys that a table can have.

In [27]:
# Join the Production.Product and Production.ProductSubcategory on `ProductSubcategoryID`.
# /scrub/
product.merge(product_subcategory,
              how='left',
              on='ProductSubcategoryID',
              suffixes=('_sub', '')
             ).head(3)

,ProductID,Name_sub,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_sub,ModifiedDate_sub,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,...,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000,NaN,NaN,NaN,NaN
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,...,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000,NaN,NaN,NaN,NaN
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,0.0,0.0,...,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000,NaN,NaN,NaN,NaN


**Exercise (5 mins., pair programming)**

- Left join `Sales.SalesOrderDetail` to `Sales.SalesOrderHeader` on the relevant primary/foreign key. (Use the data dictionaries to identify the key.)

In [28]:
# /scrub/
sales_header.merge(sales_detail,
                   how='left',
                   on='SalesOrderID'
                  ).head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid_y,ModifiedDate_y
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,1.0,4911-403C-98,1.0,776.0,1.0,2024.994,0.0,2024.994,{B207C96D-D9E6-402B-8470-2CC176C42283},2011-05-31 00:00:00
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,2.0,4911-403C-98,3.0,777.0,1.0,2024.994,0.0,6074.982,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},2011-05-31 00:00:00
2,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,3.0,4911-403C-98,1.0,778.0,1.0,2024.994,0.0,2024.994,{475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F},2011-05-31 00:00:00
3,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,4.0,4911-403C-98,1.0,771.0,1.0,2039.994,0.0,2039.994,{04C4DE91-5815-45D6-8670-F462719FBCE3},2011-05-31 00:00:00
4,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,5.0,4911-403C-98,1.0,772.0,1.0,2039.994,0.0,2039.994,{5A74C7D2-E641-438E-A7AC-37BF23280301},2011-05-31 00:00:00


- Left join `Sales.SalesOrderDetail` to `Sales.SalesOrderHeader` on the relevant primary/foreign key, and left join `Production.Product` to the result on the relevant primary/foreign key.

In [29]:
# /scrub/
(sales_header
 .merge(sales_detail,
        how='left',
        left_on='SalesOrderID',
        right_on='SalesOrderID'
       )
 .merge(product,
        how='left', 
        left_on='ProductID',
        right_on='ProductID'
       )
).head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{02935111-A546-4C6D-941F-BE12D42C158E},2014-02-08 10:01:36.827000000
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{7920BC3B-8FD4-4610-93D2-E693A66B6474},2014-02-08 10:01:36.827000000
2,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{1B486300-7E64-4C5D-A9BA-A8368E20C5A0},2014-02-08 10:01:36.827000000
3,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{CA74B54E-FC30-4464-8B83-019BFD1B2DBB},2014-02-08 10:01:36.827000000
4,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{BBFFF5A5-4BDC-49A9-A5AD-7584ADFFE808},2014-02-08 10:01:36.827000000


$\blacksquare$